> **TODO**

**Try to Predict for Another Task**

Now, your are familiar with the simple pipeline of data mining. The final task for you is to explore the ```data/dataset.xlsx``` to predict one of the labels above:
* ```patient_addmited_to_regular_ward_(1=yes,_0=no)```
* ```patient_addmited_to_semi-intensive_unit_(1=yes,_0=no)```
* ```patient_addmited_to_intensive_care_unit_(1=yes,_0=no)```

> Requirements
* Use **Row 5001-5645** of ```data/dataset.xlsx``` as the Test Split. Write code or a jupyter notebook to show how you solve this task. Please give detailed comments of your code and a good demonstration of your results.

In [1]:
# Import necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Take a glance at the provided data 

raw_data = pd.read_excel('data/dataset.xlsx', engine="openpyxl")

raw_data.info()
raw_data.describe()
raw_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5644 entries, 0 to 5643
Columns: 111 entries, Patient ID to ctO2 (arterial blood gas analysis)
dtypes: float64(70), int64(4), object(37)
memory usage: 4.8+ MB


,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.02234,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Fill in missing features

raw_data.columns = [x.lower().strip().replace(' ','_') for x in raw_data.columns]

for x in raw_data.columns:
    if raw_data[x].dtype=='float16' or  raw_data[x].dtype=='float32' or  raw_data[x].dtype=='float64':
        # raw_data[x].fillna(raw_data[x].mean())
        raw_data[x].fillna(raw_data[x].mean(skipna=True), inplace=True)

raw_data = raw_data.fillna(-999)

# Convert label-typed features
for y in raw_data.columns:
    if raw_data[y].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(raw_data[y].values))
        raw_data[y] = lbl.transform(list(raw_data[y].values))

raw_data.head(3)

,patient_id,patient_age_quantile,sars-cov-2_exam_result,"patient_addmited_to_regular_ward_(1=yes,_0=no)","patient_addmited_to_semi-intensive_unit_(1=yes,_0=no)","patient_addmited_to_intensive_care_unit_(1=yes,_0=no)",hematocrit,hemoglobin,platelets,mean_platelet_volume,...,hb_saturation_(arterial_blood_gases),pco2_(arterial_blood_gas_analysis),base_excess_(arterial_blood_gas_analysis),ph_(arterial_blood_gas_analysis),total_co2_(arterial_blood_gas_analysis),hco3_(arterial_blood_gas_analysis),po2_(arterial_blood_gas_analysis),arteiral_fio2,phosphor,cto2_(arterial_blood_gas_analysis)
0,1588,13,0,0,0,0,-2.186214e-09,-1.601319e-08,-3.535004e-10,7.438142e-09,...,-1.379737e-10,8.416397e-09,-1.655685e-09,4.139211e-10,-7.472139e-09,6.070843e-09,-2.469729e-08,4.656613e-09,6.332993e-09,5.243001e-09
1,451,17,0,0,0,0,2.365154e-01,-2.234027e-02,-5.174130e-01,1.067657e-02,...,-1.379737e-10,8.416397e-09,-1.655685e-09,4.139211e-10,-7.472139e-09,6.070843e-09,-2.469729e-08,4.656613e-09,6.332993e-09,5.243001e-09
2,3669,8,0,0,0,0,-2.186214e-09,-1.601319e-08,-3.535004e-10,7.438142e-09,...,-1.379737e-10,8.416397e-09,-1.655685e-09,4.139211e-10,-7.472139e-09,6.070843e-09,-2.469729e-08,4.656613e-09,6.332993e-09,5.243001e-09


In [15]:
# data preprocess

not_X_cols = ['patient_id','sars-cov-2_exam_result', 'patient_addmited_to_regular_ward_(1=yes,_0=no)', 'patient_addmited_to_semi-intensive_unit_(1=yes,_0=no)', 'patient_addmited_to_intensive_care_unit_(1=yes,_0=no)']
Y_cols = ['patient_addmited_to_regular_ward_(1=yes,_0=no)', 'patient_addmited_to_semi-intensive_unit_(1=yes,_0=no)', 'patient_addmited_to_intensive_care_unit_(1=yes,_0=no)']
X_cols = [x for x in raw_data.columns if x not in not_X_cols] # These columns are the features we can use to predict
X = raw_data[X_cols]
# Y_cols
Y = raw_data[Y_cols] 
np.sum(Y.to_numpy(), axis=0)

array([0.01399717, 0.00885897, 0.00726435])

In [19]:
# view y

y_num = {}
for i in range(len(Y)):
    v = tuple(Y.to_numpy()[i])
    if y_num.__contains__(v):
        y_num[v] = y_num[v] + 1
    else:
        y_num[v] = 0
print(y_num)
# y has four class

{(0, 0, 0): 5473, (0, 1, 0): 49, (1, 0, 0): 78, (0, 0, 1): 40}


In [53]:
# convert y to one-hot vectors
zero_idx = Y.loc[(Y == 0).all(axis=1)].index
# non_zero_idx = Y.loc[~(Y == 0).all(axis=1)].index
# non_zero_idx = Y.index - zero_idx
insert_col = np.zeros(Y.shape[0], dtype='int32')
insert_col[zero_idx] = 1
Y['patient_not_in_a_ward_(1=yes,_0=no)'] = insert_col
Y.head(10)

,"patient_addmited_to_regular_ward_(1=yes,_0=no)","patient_addmited_to_semi-intensive_unit_(1=yes,_0=no)","patient_addmited_to_intensive_care_unit_(1=yes,_0=no)","patient_not_in_a_ward_(1=yes,_0=no)"
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
5,0,0,0,1
6,0,0,0,1
7,0,0,0,1
8,0,1,0,0
9,0,0,0,1


In [55]:
# split dataset
X_train = X[0: 5000]
X_test = X[5000:-1]
Y_train = Y[0: 5000]
Y_test = Y[5000:-1]

In [56]:
# evaluate the model

from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, roc_curve, auc

def evaluate_model(model, name, print_or_not:bool = True, need_fit = False):
    performance = {}
    
    # cv_acc = 0
    if name == 'K Nearest Neighbors' or name == 'Random Forest' or 'Decision Tree' or need_fit:
        model.fit(X_train, Y_train)
    else:
        cv_acc = cross_val_score(lg_model, X_train, Y_train, scoring='accuracy')
        # print(cv_acc)
        # cv_acc = cv_acc.mean()
        # performance['cv_accuracy'] = cv_acc
    prds = model.predict(X_test)
    
    accuracy = accuracy_score(Y_test, prds)
    performance['accuracy'] = accuracy
    
    conf_mat = confusion_matrix(Y_test, prds)
    performance['confusion_matrix'] = conf_mat
    
    
    precision_sc = precision_score(Y_test, prds)
    performance['precision_score'] = precision_sc
    
    
    recall_sc = recall_score(Y_test, prds)
    performance['recall_score'] = recall_sc
    
    f1_sc = f1_score(Y_test, prds)
    performance['f1_score'] = f1_sc
    
    if print_or_not == True:
        print(f'evaluation of {name}:')
        # print(f'Cross Validation\'s Accuracy of {name}: {cv_acc}')
        
        print(f'Accuracy of {name}: {accuracy}')
        print(f'Confussion Matrix of {name}:\n{conf_mat}')
        print(f"Precision score of {name}: {precision_sc}")
        print(f"Recall score of {name}: {recall_sc}")
        print(f"F1 score of {name}: {f1_sc}")
    
        fpr, tpr, threshold = roc_curve(Y_test, prds)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(6,6))
        plt.title('Validation ROC')
        plt.plot(fpr, tpr, 'b', label = 'Val AUC = %0.3f' % roc_auc)
        plt.legend(loc = 'lower right')
        plt.plot([0, 1], [0, 1],'r--')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.show()
        
    return performance
    
# evaluate_model(tree, "Decision Tree")

In [60]:
from sklearn.linear_model import LogisticRegression

lg_model = LogisticRegression(penalty='l2', C=1)
lg_model.fit(X_train, Y_train)
lg_model.predict(X_test)
# evaluate_model(lg_model, "logistic Regression")

ValueError: y should be a 1d array, got an array of shape (5000, 4) instead.